# 쇼핑몰 리뷰 감성 분석 — **Full Fine-Tuning vs PEFT (LoRA)** 스켈레톤

> 버전: 2025-09-03 · 작성자: *스켈레톤 자동 생성*  
본 노트북은 **쇼핑몰 리뷰 데이터(JSON)**를 이용해 감성 분석을 수행하는 **스켈레톤**입니다.  
두 가지 학습 방식(**Full Fine-Tuning**, **PEFT/LoRA**)을 동일 파이프라인에서 비교할 수 있도록 구성되어 있습니다.

## 미션 개요
- **데이터 전처리** → **학습/검증 분할** → **토크나이징/데이터세트 구성**
- **Full FT** 학습/평가/저장
- **PEFT(LoRA)** 학습/평가/저장
- **학습 시간/정확도/저장 용량 비교** 및 간단 시각화
- 각 셀의 의도/설명을 **마크다운**으로 정리하여 리포트처럼 읽히도록 구성

> 💡 본 문서는 **스켈레톤**이므로, 필요에 맞게 **하이퍼파라미터·모델·전처리**를 조정하세요.  
> 💡 한국어 데이터에 적합한 베이스 모델(예: `klue/bert-base`)을 기본값으로 제공합니다.

## 🧭 Runbook (실행 가이드)

1. **환경 준비**
   - (로컬/Colab) 필요한 라이브러리 설치:
     ```bash
     pip install -q "transformers>=4.41.0" "datasets>=2.19.0" "peft>=0.11.0" "accelerate>=0.32.0" evaluate scikit-learn matplotlib pandas
     ```
   - GPU 사용 가능 시 `accelerate`가 알아서 최적 기기를 선택합니다.

2. **데이터 위치 지정**
   - 데이터는 **JSON 파일**이며, 필드:
     - `RawText`: 리뷰 텍스트
     - `GeneralPolarity`: 감성 레이블 (`-1`=부정, `0`=중립, `1`=긍정)
   - zip 압축이라면 노트북과 같은 폴더에 두고, 아래 **데이터 로드** 셀에서 경로(`DATA_ZIP_PATH`/`DATA_DIR`)를 설정하세요.

3. **전처리 & 분할**
   - 불용어 처리/이모지 제거 등은 **선택**입니다. 우선은 최소 전처리로 시작하세요.
   - 기본은 **3-클래스(부정/중립/긍정)** 분류. 원하면 중립을 제외해 **2-클래스**로 바꿀 수 있습니다.

4. **학습**
   - **Full FT** → **PEFT(LoRA)** 순으로 실행.
   - 각 학습 블록은 **시간 측정** 및 **메트릭**을 자동 로깅합니다.

5. **비교 & 리포트**
   - 학습 시간, 정확도/F1, 저장 용량(MB)을 표/차트로 비교.
   - 필요한 경우 WandB/MLflow 연동 섹션을 추가하세요.

## 📦 의존성 설치 (필요 시 실행)

In [ ]:

# Colab/로컬에서 필요 시 주석 해제하여 실행
# %pip install -q "transformers>=4.41.0" "datasets>=2.19.0" "peft>=0.11.0" "accelerate>=0.32.0" evaluate scikit-learn matplotlib pandas

import os, sys, math, time, json, random, shutil, pathlib, zipfile
from typing import List, Dict, Any, Optional
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
print(f"PyTorch: {torch.__version__}, CUDA available: {torch.cuda.is_available()}")

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score

import evaluate
from datasets import Dataset, DatasetDict

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    set_seed
)

from peft import LoraConfig, get_peft_model, TaskType

## ⚙️ 설정 (경로/모델/하이퍼파라미터)

In [ ]:

# === 데이터 경로 ===
DATA_ZIP_PATH = "review-sentiment-analysis.zip"   # 예: 노트북과 같은 폴더에 위치
DATA_DIR = "data"                                  # 압축 해제될 폴더

# 직접 JSON 폴더를 지정할 수도 있습니다 (zip 미사용 시)
# DATA_DIR = "/path/to/json_dir"

# JSON 파일을 찾을 글롭 패턴 (하위 폴더 포함)
GLOB_PATTERN = "**/*.json"

# 필드명 (데이터셋 스키마에 맞게 변경 가능)
TEXT_FIELD = "RawText"
LABEL_FIELD = "GeneralPolarity"

# 라벨 매핑 (3-클래스)
LABEL_MAP = {-1: 0, 0: 1, 1: 2}
ID2LABEL = {0: "negative", 1: "neutral", 2: "positive"}
LABEL2ID = {v: k for k, v in ID2LABEL.items()}

# 중립 제외하고 2-클래스(Binary)로 바꾸려면 아래 플래그 사용
BINARY_CLASS = False  # True로 두면 neutral 제외 (부정/긍정만)

# 선택적 도메인 필터 (예: ["패션", "화장품"] 등). 메타에 domain 키가 있을 때만 사용
DOMAIN_FILTER: Optional[list] = None  # 또는 ["패션", "화장품"]

# === 모델 & 토크나이저 ===
BASE_MODEL_NAME = "klue/bert-base"
MAX_LENGTH = 256

# === 학습 하이퍼파라미터 ===
SEED = 42
EPOCHS = 2
BATCH_SIZE = 16
LR_FT = 5e-5
LR_PEFT = 5e-5

# 출력/저장 경로
OUT_DIR = "outputs"
FT_DIR = os.path.join(OUT_DIR, "full_ft_model")
PEFT_DIR = os.path.join(OUT_DIR, "peft_lora_adapter")

os.makedirs(OUT_DIR, exist_ok=True)

set_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

def human_mb(bytes_size: int) -> float:
    return round(bytes_size / (1024 * 1024), 2)

def dir_size_mb(path: str) -> float:
    total = 0
    for root, _, files in os.walk(path):
        for f in files:
            fp = os.path.join(root, f)
            if os.path.isfile(fp):
                total += os.path.getsize(fp)
    return human_mb(total)

## 📥 데이터 로드 & 전처리

In [ ]:

def _extract_records_from_json(obj):
    r"""
    다양한 JSON 스키마를 최대한 견고하게 파싱하여
    [{TEXT_FIELD: str, LABEL_FIELD: int, ...(optional: domain)}] 리스트로 반환.
    """
    records = []
    if isinstance(obj, list):
        for item in obj:
            if isinstance(item, dict):
                records.append(item)
    elif isinstance(obj, dict):
        if "reviews" in obj and isinstance(obj["reviews"], list):
            for item in obj["reviews"]:
                if isinstance(item, dict):
                    records.append(item)
        else:
            records.append(obj)
    return records

def load_json_files(data_dir: str, glob_pattern: str = "**/*.json") -> pd.DataFrame:
    import glob
    paths = glob.glob(os.path.join(data_dir, glob_pattern), recursive=True)
    rows = []
    for p in paths:
        try:
            with open(p, "r", encoding="utf-8") as f:
                obj = json.load(f)
            recs = _extract_records_from_json(obj)
            for r in recs:
                if TEXT_FIELD in r and LABEL_FIELD in r:
                    rows.append({
                        "text": str(r[TEXT_FIELD]),
                        "label_raw": r[LABEL_FIELD],
                        "domain": r.get("domain") or r.get("category") or None,
                        "__src__": p
                    })
        except Exception as e:
            print(f"[WARN] JSON parse failed: {p} -> {e}")
    df = pd.DataFrame(rows)
    return df

def basic_clean(text: str) -> str:
    text = text.replace("\n", " ").strip()
    return text

# Zip 해제 (존재할 경우)
if os.path.exists(DATA_ZIP_PATH):
    print(f"Unzipping: {DATA_ZIP_PATH} -> {DATA_DIR}")
    os.makedirs(DATA_DIR, exist_ok=True)
    with zipfile.ZipFile(DATA_ZIP_PATH, 'r') as zf:
        zf.extractall(DATA_DIR)

df = load_json_files(DATA_DIR, GLOB_PATTERN)
if len(df) == 0:
    raise RuntimeError("데이터가 비어있습니다. DATA_ZIP_PATH/DATA_DIR 경로와 JSON 스키마를 확인하세요.")

if DOMAIN_FILTER:
    df = df[df["domain"].isin(DOMAIN_FILTER)]

if BINARY_CLASS:
    df = df[df["label_raw"].isin([-1, 1])].copy()
    df["label"] = (df["label_raw"] == 1).astype(int)
    ID2LABEL = {0: "negative", 1: "positive"}
    LABEL2ID = {v: k for k, v in ID2LABEL.items()}
    NUM_LABELS = 2
else:
    df["label"] = df["label_raw"].map(LABEL_MAP)
    NUM_LABELS = 3

df["text"] = df["text"].astype(str).map(basic_clean)
df = df.dropna(subset=["text", "label"]).reset_index(drop=True)

print("샘플:")
display(df.head(3))
print("라벨 분포:")
display(df["label"].value_counts(normalize=False).rename_axis("label").to_frame("count"))

## 🔀 학습/검증 분할

In [ ]:

train_df, test_df = train_test_split(
    df, 
    test_size=0.2, 
    random_state=SEED, 
    stratify=df["label"]
)
print(f"train={len(train_df)}, test={len(test_df)}")

train_ds = Dataset.from_pandas(train_df[["text","label"]], preserve_index=False)
test_ds  = Dataset.from_pandas(test_df[["text","label"]], preserve_index=False)
raw_datasets = DatasetDict({"train": train_ds, "test": test_ds})
raw_datasets

## 🔡 토크나이저 & 데이터셋 구성

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME, use_fast=True)

def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LENGTH
    )

tok_datasets = raw_datasets.map(tokenize_fn, batched=True, remove_columns=["text"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tok_datasets

## 🧠 Baseline: Full Fine-Tuning

In [ ]:

ft_model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL_NAME, 
    num_labels=NUM_LABELS, 
    id2label=ID2LABEL, 
    label2id=LABEL2ID
)

metric_accuracy = evaluate.load("accuracy")
metric_f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = metric_accuracy.compute(predictions=preds, references=labels)["accuracy"]
    f1_macro = metric_f1.compute(predictions=preds, references=labels, average="macro")["f1"]
    return {"accuracy": acc, "f1_macro": f1_macro}

ft_args = TrainingArguments(
    output_dir=os.path.join(OUT_DIR, "ft_tmp"),
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR_FT,
    num_train_epochs=EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to=[],
    seed=SEED
)

ft_trainer = Trainer(
    model=ft_model,
    args=ft_args,
    train_dataset=tok_datasets["train"],
    eval_dataset=tok_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

t0 = time.perf_counter()
ft_train_out = ft_trainer.train()
ft_train_sec = time.perf_counter() - t0

ft_eval = ft_trainer.evaluate()
print("Full FT eval:", ft_eval)

# 저장
if os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR, exist_ok=True)
if os.path.exists("ft_tmp"):
    pass

if os.path.exists(FT_DIR):
    import shutil as _shutil
    _shutil.rmtree(FT_DIR)
ft_trainer.save_model(FT_DIR)
tokenizer.save_pretrained(FT_DIR)

ft_size_mb = dir_size_mb(FT_DIR)
print(f"[Full FT] 저장 용량: {ft_size_mb} MB")

## 🔧 PEFT (LoRA) 학습

In [ ]:

peft_base = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL_NAME, 
    num_labels=NUM_LABELS, 
    id2label=ID2LABEL, 
    label2id=LABEL2ID
)

target_modules = ["query", "key", "value", "dense"]

lora_cfg = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=target_modules
)

peft_model = get_peft_model(peft_base, lora_cfg)
peft_model.print_trainable_parameters()

peft_args = TrainingArguments(
    output_dir=os.path.join(OUT_DIR, "peft_tmp"),
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR_PEFT,
    num_train_epochs=EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to=[],
    seed=SEED
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_args,
    train_dataset=tok_datasets["train"],
    eval_dataset=tok_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

t0 = time.perf_counter()
peft_train_out = peft_trainer.train()
peft_train_sec = time.perf_counter() - t0

peft_eval = peft_trainer.evaluate()
print("PEFT eval:", peft_eval)

# 어댑터만 저장
import shutil as _shutil
if os.path.exists(PEFT_DIR):
    _shutil.rmtree(PEFT_DIR)
peft_trainer.model.save_pretrained(PEFT_DIR)
tokenizer.save_pretrained(PEFT_DIR)

peft_size_mb = dir_size_mb(PEFT_DIR)
print(f"[PEFT] 저장 용량(어댑터): {peft_size_mb} MB")

## 📊 두 방식 비교 (시간 · 정확도 · 용량)

In [ ]:

import pandas as _pd
summary = _pd.DataFrame([
    {
        "approach": "Full FT",
        "train_sec": round(ft_train_sec, 2),
        "accuracy": round(float(ft_eval.get("eval_accuracy", float("nan"))), 4),
        "f1_macro": round(float(ft_eval.get("eval_f1_macro", float("nan"))), 4),
        "saved_size_mb": ft_size_mb
    },
    {
        "approach": "PEFT (LoRA)",
        "train_sec": round(peft_train_sec, 2),
        "accuracy": round(float(peft_eval.get("eval_accuracy", float("nan"))), 4),
        "f1_macro": round(float(peft_eval.get("eval_f1_macro", float("nan"))), 4),
        "saved_size_mb": peft_size_mb
    }
])

display(summary)

import matplotlib.pyplot as _plt

_plt.figure()
_plt.bar(summary["approach"], summary["saved_size_mb"])
_plt.title("저장 용량 비교 (MB)")
_plt.ylabel("MB")
_plt.show()

_plt.figure()
_plt.bar(summary["approach"], summary["accuracy"])
_plt.title("정확도(Accuracy) 비교")
_plt.ylabel("accuracy")
_plt.ylim(0, 1)
_plt.show()

## 📝 참고 & 트러블슈팅

- **JSON 스키마가 다른 경우**: `TEXT_FIELD`, `LABEL_FIELD`, `_extract_records_from_json()` 수정
- **2-클래스(부정/긍정)만 사용**: `BINARY_CLASS=True` 설정
- **PEFT 타깃 모듈 에러**: 모델 구조에 맞게 `target_modules` 수정 (예: `query_proj`, `value_proj` 등)
- **속도 개선**: `fp16=True`(AMP), `gradient_accumulation_steps`, 짧은 `MAX_LENGTH` 등
- **추가 지표**: precision/recall, confusion matrix 등 필요 시 확장
- **모델 병합 저장**: `peft_model.merge_and_unload()` 사용 가능

## ✅ (선택) 미니 샌티티 체크

In [ ]:

# 전체 학습 전에 파이프라인 점검용 소량 데이터로 1 epoch만 빠르게 검증하고 싶다면
# 아래 예시를 참고해 주석 해제/수정하여 사용하세요.

# mini_train = tok_datasets["train"].select(range(min(256, len(tok_datasets["train"]))))
# mini_test  = tok_datasets["test"].select(range(min(256, len(tok_datasets["test"]))))

# args_quick = TrainingArguments(
#     output_dir=os.path.join(OUT_DIR, "quick_tmp"),
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     learning_rate=5e-5,
#     num_train_epochs=1,
#     evaluation_strategy="epoch",
#     logging_steps=20,
#     report_to=[],
#     seed=SEED
# )
# quick_model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL_NAME, num_labels=NUM_LABELS)
# quick_tr = Trainer(
#     model=quick_model, args=args_quick,
#     train_dataset=mini_train, eval_dataset=mini_test,
#     tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics
# )
# quick_tr.train(); print(quick_tr.evaluate())